In [ ]:
import pandas as pd
df = pd.read_excel('/content/Violence-Project-Mass-Shooter-Database-Version-5-May-2022 (1).xlsx')
df

,The Violence Project Mass Shooter Database,Unnamed: 1,Unnamed: 2
0,Version 5.0 (Updated May 2022),NaN,NaN
1,"Principal Investigators: Jillian Peterson, PhD...",NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,Worksheet,Title,Description
5,Worksheet 1,Table of Contents,NaN
6,Worksheet 2,Frequently Asked Questions - READ THIS FIRST,"Terms of usage, definition of mass shootings, ..."
7,Worksheet 3,New in Each Version,List of updates to the database from Version 1...
8,Worksheet 4,Codebook,Variable codes and definitions
9,Worksheet 5,Full Database,179 mass shooters from 1966 to 2022 coded on o...


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
XLSX_PATH = '/content/Violence-Project-Mass-Shooter-Database-Version-5-May-2022 (1).xlsx'
FIREARMS_SHEET = 'Firearms Data'      # per your index row 10
FULL_DB_SHEET  = 'Full Database'      # optional (for fatalities/outcomes if you want)
FIGDIR = 'figs'

In [ ]:
fire = pd.read_excel(XLSX_PATH, sheet_name=FIREARMS_SHEET)

# Quick peek
print("Firearms sheet shape:", fire.shape)
print("Columns:", list(fire.columns)[:30])

# ---------- Normalize column names (robust to slight naming changes) ----------
def norm(s):
    return re.sub(r'\s+', ' ', str(s)).strip().lower()

fire_cols = {norm(c): c for c in fire.columns}

def pick(*cands):
    for cand in cands:
        # exact normalized match
        if cand in fire_cols:
            return fire_cols[cand]
        # substring fallback
        for k,v in fire_cols.items():
            if cand in k:
                return v
    return None

# Common columns found in TVP firearms sheet (names can vary slightly by version)
# Adjust these if your file uses different labels.
incident_id_col   = pick('case number', 'incident id', 'case', 'case id', 'id')
weapon_type_col   = pick('weapon type', 'firearm type', 'type of weapon', 'weapon')
acq_method_col    = pick('acquisition', 'acquisition method', 'obtained', 'purchase', 'source')
legality_col      = pick('legal purchase', 'legality', 'legal/illegal', 'was purchase legal')
assault_flag_col  = pick('assault rifle', 'ar-15', 'assault weapon', 'assault')
highcap_flag_col  = pick('high-capacity', 'high capacity', 'large capacity', 'magazine capacity')
num_weapons_col   = pick('total weapons', 'number of weapons', 'weapons brought', 'num weapons')
semi_auto_col     = pick('semi-auto', 'semiautomatic', 'semi automatic')
year_col          = pick('year', 'date')  # if the sheet has year/date; optional

# Coerce some to numeric/bools if present
for c in [num_weapons_col]:
    if c is not None:
        fire[c] = pd.to_numeric(fire[c], errors='coerce')

def to_bool_col(s):
    if s is None:
        return None
    # Normalize common yes/no / true/false text
    return fire[s].astype(str).str.strip().str.lower().map({
        '1': True, 'true': True, 'yes': True, 'y': True, 't': True,
        '0': False, 'false': False, 'no': False, 'n': False, 'f': False
    })

assault_bool = to_bool_col(assault_flag_col)
highcap_bool = to_bool_col(highcap_flag_col)

# ---------- Helper to save a figure ----------
def savefig(name):
    plt.tight_layout()
    path = os.path.join(FIGDIR, name)
    plt.savefig(path, dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved:", path)

Firearms sheet shape: (391, 17)
Columns: ['Case #', 'Shooter Last Name', 'Shooter First Name', 'Full Date', 'Make and Model', 'Classification', 'Caliber', 'Used in Shooting?', 'Modified', 'Extended Magazine', 'When Obtained', 'Legal Purchase', 'Illegal Purchase', 'Assembled with Legal Parts ', 'Gifted', 'Theft', 'Unknown']


In [ ]:
if weapon_type_col:
    vc = (fire[weapon_type_col]
          .astype(str).str.strip().str.title()
          .replace({'Nan':'Unknown', '':'Unknown'})
          .value_counts(dropna=False)
          .sort_values(ascending=False))
    plt.figure(figsize=(8,4))
    vc.plot(kind='bar')
    plt.title('Weapon Type Distribution')
    plt.xlabel('Weapon Type')
    plt.ylabel('Count')
    plt.show()
    savefig('weapon_type_distribution.png')

In [ ]:
file_path = '/content/Violence-Project-Mass-Shooter-Database-Version-5-May-2022 (1).xlsx'
df = pd.read_excel(file_path, sheet_name='Firearms Data')

# Clean column names
df.columns = df.columns.str.strip()

# Create output directory for LaTeX figures
os.makedirs("figs", exist_ok=True)

# Convert 'Full Date' to datetime and extract year
df['Full Date'] = pd.to_datetime(df['Full Date'], errors='coerce')
df['Year'] = df['Full Date'].dt.year

# 1️⃣ Weapon Classification Distribution
plt.figure(figsize=(8,4))
df['Classification'].value_counts().plot(kind='bar')
plt.title("Distribution of Firearm Classifications")
plt.xlabel("Classification")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("figs/firearm_classification_distribution.png", dpi=300)
plt.close()

In [ ]:
plt.figure(figsize=(8,4))
df['Caliber'].value_counts().head(10).plot(kind='bar')
plt.title("Top 10 Firearm Calibers")
plt.xlabel("Caliber")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("figs/firearm_caliber_distribution.png", dpi=300)
plt.close()

In [ ]:
plt.figure(figsize=(5,5))
df['Used in Shooting?'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title("Firearms Used in Shooting")
plt.ylabel("")
plt.tight_layout()
plt.savefig("figs/firearm_used_pie.png", dpi=300)
plt.close()


In [ ]:
plt.figure(figsize=(5,5))
df['Modified'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title("Modified Firearms Proportion")
plt.ylabel("")
plt.tight_layout()
plt.savefig("figs/firearm_modified_pie.png", dpi=300)
plt.close()

In [ ]:
plt.figure(figsize=(5,5))
df["Extended Magazine"].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title("Extended Magazine Presence")
plt.ylabel("")
plt.tight_layout()
plt.savefig("figs/extended_magazine_pie.png", dpi=300)
plt.close()

In [ ]:
plt.figure(figsize=(8,4))
df["When Obtained"].value_counts().head(10).plot(kind='bar', color='slateblue')
plt.title("When Firearms Were Obtained (Top 10 categories)")
plt.xlabel("Timeframe Before Incident")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("figs/when_obtained_bar.png", dpi=300)
plt.close()

In [ ]:
legality_cols = ["Legal Purchase", "Illegal Purchase", "Assembled with legal parts",
                 "Gifted", "Theft", "Unknown"]
available_cols = [c for c in legality_cols if c in df.columns]

legality_summary = df[available_cols].apply(lambda x: (x == "Yes").sum())
plt.figure(figsize=(8,4))
legality_summary.plot(kind='bar', color='darkorange')
plt.title("Firearm Acquisition Methods")
plt.xlabel("Acquisition Type")
plt.ylabel("Number of Firearms")
plt.tight_layout()
plt.savefig("figs/acquisition_legality_bar.png", dpi=300)
plt.close()

In [ ]:
if df["Year"].notna().sum() > 0:
    yearly_counts = df["Year"].value_counts().sort_index()
    plt.figure(figsize=(8,4))
    plt.plot(yearly_counts.index, yearly_counts.values, marker='o', color='firebrick')
    plt.title("Firearms Recorded per Year")
    plt.xlabel("Year")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig("figs/firearm_trend_by_year.png", dpi=300)
    plt.close()

# 9️⃣ Top firearm make and model
plt.figure(figsize=(8,4))
df["Make and Model"].value_counts().head(10).plot(kind='bar', color='seagreen')
plt.title("Top 10 Firearm Make & Models")
plt.xlabel("Make & Model")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("figs/top_firearm_make_model.png", dpi=300)
plt.close()